In [1]:
import nltk as nl
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import tweepy
import csv

## Análise de sentimento no twitter

Utilizando uma base de tweets em português já classificados como positivos ou negativos, um classificador do tipo Naive Bayes será treinado para depois ser aplicado em um conjunto de tweets específico.

###### Leitura e visualização dos dados

In [2]:
df = pd.read_csv('db.csv', encoding="utf-8", sep='\t')
df.head()

,id,text,sentiment
0,1,@caprichOreality Fica assim não miga &lt;3 Tud...,1
1,2,Parti me todo a descer a avenida de Gaia com o...,1
2,3,Amanhã é dia de dar um trato na palestra para ...,1
3,4,@thankovsky @patorebaichado eu também tenho :)...,1
4,5,ok. Sim. Aham. Tá. De boa. Vai lá. :) https://...,1


In [3]:
df[df.sentiment == 0].count()

id           28172
text         28172
sentiment    28172
dtype: int64

28172 tweets classificados como negativos (0)

In [4]:
df[df.sentiment == 1].count()

id           29924
text         29924
sentiment    29924
dtype: int64

29924 tweets classificados como positivos (1)

###### Limpeza dos dados

In [5]:
def clean_mentions(text):
    return re.sub(r"@\w+", "", text)

In [6]:
def clean_links(text):
    return re.sub(r"http\S+", "", text)

In [7]:
def clean_stopwords(text):
    stopwords = set(nl.corpus.stopwords.words('portuguese'))
    words = [i for i in text.split() if not i in stopwords]
    return (" ".join(words))

In [8]:
def clean_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text_with_no_special_chars = re.sub("\s+", " ", text)
    return text_with_no_special_chars

In [9]:
def clean_text(text):
    text = text.lower()
    text = clean_links(text)
    text = clean_mentions(text)
    text = clean_special_chars(text)
    text = clean_stopwords(text)
    return text

In [10]:
for index, row in df.iterrows():
    df.at[index, 'text'] = clean_text(df.at[index, 'text'])

In [11]:
df.head()

,id,text,sentiment
0,1,fica assim miga lt tudo arranja deus quiser,1
1,2,parti todo descer avenida gaia skate,1
2,3,amanhã é dia dar trato palestra thedevconf aju...,1
3,4,posso sentar,1
4,5,ok sim aham tá boa vai lá,1


Os tweets de treinamento após a limpeza.

###### Modelo

Usando a abordagem Bag of Words e o algoritmo Naive Bayes

In [13]:
tweets = df["text"].values

In [14]:
classes = df["sentiment"].values

In [15]:
'''
Cria um vetor com cada uma das palavras do texto completo da base, 
depois, calcula a frequência em que essas palavras ocorrem em uma data sentença, para então classificar/treinar o modelo
'''
vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

'''
Ajusta o modelo, aprende o vocabulário, e transforma os dados de treinamento em vetores com frequêcia das palavras
'''
modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)

In [17]:
metrics.accuracy_score(classes, resultados)

0.7437689341779125

Acurácia de 0.74

Verificando as medidas de validação do modelo:

In [18]:
sentimentos = [1, 0]
print(metrics.classification_report(classes, resultados, sentimentos))

             precision    recall  f1-score   support

          1       0.77      0.71      0.74     29924
          0       0.72      0.78      0.75     28172

avg / total       0.75      0.74      0.74     58096



Em que:
* precision = true positive / (true positive + false positive)
* recall    = true positive / (true positive + false negative)
* f1-score  = 2 x ((precision x recall) / (precision + recall))

O algoritmo apresentou precisão de 0.77 ao classificar tweets positivos e 0.72 ao classificar tweets negativos, e uma precisão geral de 0.75.

In [19]:
print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames=["Predito"], margins=True))

Predito      0      1    All
Real                        
0        21957   6215  28172
1         8671  21253  29924
All      30628  27468  58096


Em que:
* Predito = O que o programa classificou como Negativo, Positivo e All
* Real    = O que é de fato Negativo, Positivo e All

Ou seja, o algoritmo classificou 21957 tweets realmente negativos como negativos, e 6215 tweets realmente negativos como positivos.

### Aplicação em dados da hashtag MasterChefBR

O classificador treinado será utilizado para analisar o sentimento de tweets contendo a hashtag #MasterChefBR. Os dados são baixados utilizando credenciais da api do Twitter. Como são muitos tweets e é necessário fazer uma classificação manual do sentimento, serão escolhidos e classificados 50 tweets.

In [20]:
####input your credentials here
consumer_key = 'X1iKxRO9vW3VBdfdQrBxCKneA'
consumer_secret = 'J73REOsIZoWFnjPC0VZtF3CWRTBrfx0l4v85TvAUC8tBIJ1PwA'
access_token = '191642356-02GI0AIT8ddbmWuDOF6twcMU0sRzyoiyjiX6bvtZ'
access_token_secret = 'QWMtL7jSYMgR1Rk5gQf8pI09cRqchb2jE7MUcFYXuatJD'

In [21]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# Open/Create a file to append data
csvFile = open('mc.csv', 'w')
#Use csv Writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(["Data", "Tweet"])

for tweet in tweepy.Cursor(api.search,q="#MasterChefBR -RT",count=100,
                           lang="pt",
                           since="2018-01-01").items():
    text = clean_text(tweet.text)
    csvWriter.writerow([tweet.created_at, text.encode('utf-8')])

In [22]:
tweets = pd.read_csv("mc.csv", encoding="utf-8")
tweets.head()

,Data,Tweet,sentiment
0,2018-07-31 02:08:33,bora lá hugo pega troféu masterchefbr ganhahugo,1
1,2018-07-31 02:02:44,segunda feira 23h jovem pensa começa final mas...,1
2,2018-07-31 01:52:33,embuste ganhar amanhã vou ficar putassa master...,0
3,2018-07-31 01:52:10,hugo ganhar nada importa doq opinião masterchefbr,1
4,2018-07-31 01:42:48,maria antônia diabo suporta masterchefbr,0


Os tweets já selecionados e classificados.

In [23]:
testes = []
for index, row in tweets.iterrows():
    testes.append(tweets.at[index, 'Tweet'])

In [24]:
classes_teste = tweets["sentiment"].values

In [25]:
freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1], dtype=int64)

In [26]:
resultados_teste = cross_val_predict(modelo, freq_testes, classes_teste, cv = 10)

In [27]:
metrics.accuracy_score(classes_teste, resultados_teste)

0.7

Acurácia de 0.7, apenas 0.04 menor que a do treinamento, devido ao número de tweets ser bem menor.

In [28]:
sentimentos = [0, 1]
print(metrics.classification_report(classes_teste, resultados_teste, sentimentos))

             precision    recall  f1-score   support

          0       0.76      0.54      0.63        24
          1       0.67      0.85      0.75        26

avg / total       0.71      0.70      0.69        50



O algoritmo apresentou precisão de 0.67 ao classificar tweets positivos e 0.76 ao classificar tweets negativos, e uma precisão geral de 0.71. A precisão foi menor que a do treinamento, exceto na classificação de tweets negativos.

In [29]:
print(pd.crosstab(classes_teste, resultados_teste, rownames = ["Real"], colnames=["Predito"], margins=True))

Predito   0   1  All
Real                
0        13  11   24
1         4  22   26
All      17  33   50


Ou seja, o algoritmo classificou 13 tweets realmente negativos como negativos, e 11 tweets realmente negativos como positivos. Para o caso dos tweets realmente positivos o algoritmo apresentou melhor predição, de 22 tweets positivos, quando o total é de 26.